# Classes and functions from FairSMOTE

In [ ]:
from __future__ import print_function, division
import pdb
import unittest
import random
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors as NN

class smote(object):
  def __init__(self, pd_data, neighbor=5,r=2 ,up_to_num=[],auto=True):
    """
    :param pd_data: panda.DataFrame, the last column must be class label
    :param neighbor: num of nearst neighbors to select
    :param up_to_num: size of minorities to over-sampling
    :param up_to_max: if up_to_num is not supplied, all minority classes will
                      be over-sampled as much as majority class
    :return panda.DataFrame smoted data
    """
    self.set_data(pd_data)
    self.auto = auto
    self.neighbor = neighbor
    self.up_to_max = False
    self.up_to_num = up_to_num
    self.r = r
    self.label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #if up_to_num:
    #  label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #  if label_num - 1 != len(up_to_num):
    #    raise ValueError(
    #      "should set smoted size for " + str(label_num - 1) + " minorities")
    #  self.up_to_num = up_to_num
    #else:
    #  self.up_to_max = True

  def set_data(self, pd_data):
    if not pd_data.empty:# and isinstance(
        #pd_data.ix[:, pd_data.columns[-1]].values[0], str):
      self.data = pd_data
    else:
      raise ValueError(
        "The last column of pd_data should be string as class label")

  def get_majority_num(self):
    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    return majority_num

  def run(self):
    """
    run smote
    """

    def get_ngbr(data_no_label, knn):
      rand_sample_idx = random.randint(0, len(data_no_label) - 1)
      rand_sample = data_no_label[rand_sample_idx]
      distance, ngbr = knn.kneighbors(rand_sample.reshape(1, -1))
      # while True:
      rand_ngbr_idx = random.randint(0, len(ngbr))
      #   if distance[rand_ngbr_idx] == 0:
      #     continue  # avoid the sample itself, where distance ==0
      #   else:
      return data_no_label[rand_ngbr_idx], rand_sample

    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    for label, num in labelCont.items():
      if num < majority_num:
        to_add = majority_num - num
        last_column = self.data[self.data.columns[-1]]
        data_w_label = self.data.loc[last_column == label]
        data_no_label = data_w_label[self.data.columns[:-1]].values
        if len(data_no_label) < self.neighbor:
          num_neigh = len(data_no_label) # void # of neighbors >= sample size
        else:
          num_neigh = self.neighbor
        knn = NN(n_neighbors=num_neigh,p=self.r,algorithm='ball_tree').fit(data_no_label)
        if self.auto:
          to_add = to_add
        else:
          to_add = self.up_to_num
        for _ in range(to_add):
          rand_ngbr, sample = get_ngbr(data_no_label, knn)
          new_row = []
          for i, one in enumerate(rand_ngbr):
            gap = random.random()
            new_row.append(max(0, sample[i] + (
            sample[i] - one) * gap))  # here, feature vlaue should >=0
          new_row.append(label)
          total_data.append(new_row)
    return pd.DataFrame(total_data)

In [ ]:
from __future__ import print_function, division
import pdb
import unittest
import random
from collections import Counter
import pandas as pd
import numpy as np
from scipy.spatial import distance as dist
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors as NN

def get_ngbr(df, knn):
            rand_sample_idx = random.randint(0, df.shape[0] - 1)
            parent_candidate = df.iloc[rand_sample_idx]
            ngbr = knn.kneighbors(parent_candidate.values.reshape(1,-1),3,return_distance=False)
            candidate_1 = df.iloc[ngbr[0][0]]
            candidate_2 = df.iloc[ngbr[0][1]]
            candidate_3 = df.iloc[ngbr[0][2]]
            return parent_candidate,candidate_2,candidate_3

def generate_samples(no_of_samples,df,df_name):

    total_data = df.values.tolist()
    knn = NN(n_neighbors=5,algorithm='auto').fit(df)

    for _ in range(no_of_samples):
        cr = 0.8
        f = 0.8
        parent_candidate, child_candidate_1, child_candidate_2 = get_ngbr(df, knn)
        new_candidate = []
        for key,value in parent_candidate.items():
            if isinstance(parent_candidate[key], bool):
                new_candidate.append(parent_candidate[key] if cr < random.random() else not parent_candidate[key])
            elif isinstance(parent_candidate[key], str):
                new_candidate.append(random.choice([parent_candidate[key],child_candidate_1[key],child_candidate_2[key]]))
            elif isinstance(parent_candidate[key], list):
                temp_lst = []
                for i, each in enumerate(parent_candidate[key]):
                    temp_lst.append(parent_candidate[key][i] if cr < random.random() else
                                    int(parent_candidate[key][i] +
                                        f * (child_candidate_1[key][i] - child_candidate_2[key][i])))
                new_candidate.append(temp_lst)
            else:
                new_candidate.append(abs(parent_candidate[key] + f * (child_candidate_1[key] - child_candidate_2[key])))
        total_data.append(new_candidate)

    final_df = pd.DataFrame(total_data)
    if df_name == 'Adult':
        final_df = final_df.rename(columns={0:"age",1:"education.num",2:"race",3:"sex",4:"capital.gain",5:"capital.loss",6:"hours.per.week",7:"income"}, errors="raise")
    if df_name == 'Compas':
        final_df = final_df.rename(columns={0:"age",1:"c_charge_degree",2:"age_cat",3:"sex",4:"priors_count",5:"days_b_screening_arrest",6:"decile_score", 7:"am", 8:"two_year_recid"}, errors="raise")
    if df_name == 'Default':
    	final_df = final_df.rename(columns={0:"ID",1:"LIMIT_BAL",2:"sex",3:"EDUCATION",4:"MARRIAGE",5:"AGE",6:"PAY_0",7:"PAY_2",8:"PAY_3",9:"PAY_4",10:"PAY_5",11:"PAY_6",12:"BILL_AMT1",13:"BILL_AMT2",14:"BILL_AMT3",15:"BILL_AMT4",16:"BILL_AMT5",17:"BILL_AMT6",18:"PAY_AMT1",19:"PAY_AMT2",20:"PAY_AMT3",21:"PAY_AMT4",22:"PAY_AMT5",23:"PAY_AMT6",24:"Probability"}, errors="raise")
    if df_name == 'German':
    	final_df = final_df.rename(columns={0:"sex",1:"age",2:"Probability",3:"credit_history=Delay",4:"credit_history=None/Paid",5:"credit_history=Other",6:"savings=500+",7:"savings=<500",8:"savings=Unknown/None",9:"employment=1-4 years",10:"employment=4+ years",11:"employment=Unemployed"}, errors="raise")
    if df_name == 'Heart':
    	final_df = final_df.rename(columns={0:"age",1:"sex",2:"cp",3:"trestbps",4:"chol",5:"fbs",6:"restecg",7:"thalach",8:"exang",9:"oldpeak",10:"slope",11:"ca",12:"thal",13:"Probability"}, errors="raise")
    if df_name == 'Bank':
        final_df = final_df.rename(columns={0:"age",1:"default",2:"balance",3:"housing",4:"loan",5:"day",6:"duration",7:"campaign",8:"pdays",9:"previous",10:"Probability"}, errors="raise")
    if df_name == 'Titanic':
        final_df = final_df.rename(columns={0:"Pclass",1:"sex",2:"Age",3:"SibSp",4:"Parch",5:"Fare",6:"Probability"}, errors="raise")
    if df_name == 'Student':
        final_df = final_df.rename(columns={0:'sex', 1:'age', 2:'Medu', 3:'Fedu', 4:'traveltime', 5:'studytime', 6:'failures',
       7:'schoolsup', 8:'famsup', 9:'paid', 10:'activities', 11:'nursery', 12:'higher',
       13:'internet', 14:'romantic', 15:'famrel', 16:'freetime', 17:'goout', 18:'Dalc', 19:'Walc',
       20:'health', 21:'absences', 22:'G1', 23:'G2', 24:'Probability'}, errors="raise")

    return final_df

In [ ]:
import time
import pandas as pd
import random,time,csv
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sklearn.metrics as metrics


import sys
sys.path.append(os.path.abspath('..'))
from imblearn import under_sampling, over_sampling

# from SMOTE import smote
# from imblearn.over_sampling import SMOTE

# from Measure import measure_final_score,calculate_recall,calculate_far,calculate_precision,calculate_accuracy
# from Generate_Samples import generate_samples



!pip uninstall numpy
!pip install numpy
! pip install ctgan
! pip install --upgrade category_encoders


# ! pip install table_evaluator

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import category_encoders as ce
import math
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_auc_score, confusion_matrix
import os
from statistics import mean
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from datetime import datetime
import pprint
import json



Found existing installation: numpy 1.25.1
Uninstalling numpy-1.25.1:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.10
    /usr/local/lib/python3.10/dist-packages/numpy-1.25.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libopenblas64_p-r0-7a851222.3.23.so
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.10/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-1.25.1
  Using cached numpy-1.25.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24

In [ ]:
# Record the start time
start_time = time.time()

# Load Dataset

In [ ]:
# ## Load dataset
# from sklearn import preprocessing
# dataset_orig = pd.read_csv('../data/adult.data.csv')

# ## Drop NULL values
# dataset_orig = dataset_orig.dropna()

# ## Drop categorical features
# dataset_orig = dataset_orig.drop(['workclass','fnlwgt','education','marital-status','occupation','relationship','native-country'],axis=1)

# ## Change symbolics to numerics
# dataset_orig['sex'] = np.where(dataset_orig['sex'] == ' Male', 1, 0)
# dataset_orig['race'] = np.where(dataset_orig['race'] != ' White', 0, 1)
# dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == ' <=50K', 0, 1)


# ## Discretize age
# dataset_orig['age'] = np.where(dataset_orig['age'] >= 70, 70, dataset_orig['age'])
# dataset_orig['age'] = np.where((dataset_orig['age'] >= 60 ) & (dataset_orig['age'] < 70), 60, dataset_orig['age'])
# dataset_orig['age'] = np.where((dataset_orig['age'] >= 50 ) & (dataset_orig['age'] < 60), 50, dataset_orig['age'])
# dataset_orig['age'] = np.where((dataset_orig['age'] >= 40 ) & (dataset_orig['age'] < 50), 40, dataset_orig['age'])
# dataset_orig['age'] = np.where((dataset_orig['age'] >= 30 ) & (dataset_orig['age'] < 40), 30, dataset_orig['age'])
# dataset_orig['age'] = np.where((dataset_orig['age'] >= 20 ) & (dataset_orig['age'] < 30), 20, dataset_orig['age'])
# dataset_orig['age'] = np.where((dataset_orig['age'] >= 10 ) & (dataset_orig['age'] < 10), 10, dataset_orig['age'])
# dataset_orig['age'] = np.where(dataset_orig['age'] < 10, 0, dataset_orig['age'])

# protected_attribute = 'race'

# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)


# dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2,shuffle = True)

# # dataset_orig

## Fairness metrics and functions


In [ ]:
def print_metrics(y_gt, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_gt, y_pred).ravel()
    print('conf matrix:\n ', confusion_matrix(y_gt, y_pred) ,
          '\naccuracy: ', (tp+tn)/(tp+tn+fn+fp),
          'precision: ', tp/(tp+fp),
          'recall: ', tp/(tp+fn),
          'fpr: ', fp/(fp+tn),
          'tpr: ', tp /(tp + fn),
          'fn+fp', fn+fp)
    #returm fpr, tpr, accuracy
    return fp/(fp+tn), tp /(tp + fn), (tp+tn)/(tp+tn+fn+fp)

# bias metric - equal opportunity difference
def opportunity_diff_tpr(tpr1, tpr2):
    return abs(tpr1-tpr2)

# bias metric - equal opportunity difference
def opportunity_diff_fpr(fpr1, fpr2):
    return abs(fpr1-fpr2)

# bias metric - avarage absolute odds difference
def odds_diff(tpr1, tpr2, fpr1, fpr2):
    return 0.5*(abs(tpr2-tpr1) + abs(fpr2-fpr1))

# bias metric - statistical parity difference
def parity_diff(y_pred_0, y_pred_1):
    return abs(len(y_pred_0)/(len(y_pred_0)+len(y_pred_1)) - len(y_pred_1)/(len(y_pred_1)+len(y_pred_0)))


    # bias metric - Disparate impact
def disparate_impact(y_pred_0, y_pred_1):
    return min((len(y_pred_0)/len(y_pred_0+y_pred_1)) / (len(y_pred_1)/len(y_pred_1+len(y_pred_0)))
    ,(len(y_pred_1)/len(y_pred_0+y_pred_1)) / (len(y_pred_0)/len(y_pred_1+len(y_pred_0))))

In [ ]:
# calculate the bias of divided dataset by the privileged feature
def calc_bias(data,y, priv_feature,y_pred, experiment_text):
    data_full =  pd.concat([data, y], axis=1)
    data_full['y_pred'] = y_pred

    label = TARGET_COL
    print('performance for whole ds:'+ experiment_text)
    fpr, tpr, acc = print_metrics(data_full[TARGET_COL], data_full['y_pred'] )

    feature_cat_list = list(data[priv_feature].unique())
    #print(feature_cat_list)
    valid_cat0 = data_full[data_full[priv_feature]==feature_cat_list[0]]
    valid_cat1 = data_full[data_full[priv_feature]==feature_cat_list[1]]
    print('performance for 0 sub-group:')
    fpr_0, tpr_0 , acc_0 = print_metrics(valid_cat0[TARGET_COL], valid_cat0['y_pred'])
    print('performance for 1 sub-group:')

    fpr_1, tpr_1, acc_1 = print_metrics(valid_cat1[TARGET_COL], valid_cat1['y_pred'])
    op_diff = opportunity_diff_tpr(tpr_0, tpr_1)
    op_diff_fpr = opportunity_diff_fpr(fpr_0, fpr_1)
    od_diff = odds_diff(tpr_0, tpr_1, fpr_0, fpr_1)
    par_diff = parity_diff(valid_cat0[valid_cat0['y_pred']==1], valid_cat1[valid_cat1['y_pred']==1])
    dis_impact = disparate_impact(valid_cat0[valid_cat0['y_pred']==1], valid_cat1[valid_cat1['y_pred']==1])
    print('bias metrics:')
    print('equal opportunity for tpr: (close to 0)', op_diff)
    print('equal opportunity for fpr: (close to 0)', op_diff_fpr)
    print('avarage absolute odds difference: (close to 0)',od_diff)
    print('statistical parity difference: (close to 0) ',par_diff)
    print('disparate_impact: (close to 1) ',dis_impact)

#     # Plot for orig_data_one_priv.shape[0] and orig_data_one_non_priv.shape[0]
#     fig, axs = plt.subplots(1, 2, figsize=(12, 6))

#     # Bar colors
#     colors = ['blue', 'orange']

#     # Plot 1 - orig_data_one_priv and orig_data_one_non_priv
#     axs[0].bar(['priv', 'non_priv'], [tpr_0, tpr_1], color=colors)
#     axs[0].set_xlabel('Data Type')
#     axs[0].set_ylabel('Count')
#     axs[0].set_title('TPR(maximize)')

#     # Plot 2 - orig_data_zero_priv and orig_data_zero_non_priv
#     axs[1].bar(['priv', 'non_priv'], [fpr_0, fpr_1], color=colors)
#     axs[1].set_xlabel('Data Type')
#     axs[1].set_ylabel('Count')
#     axs[1].set_title('FPR(minimize)')

#     # Adjust spacing between subplots
#     plt.subplots_adjust(wspace=0.4)

#     # Show the plots
#     plt.show()

    return (op_diff,op_diff_fpr, od_diff,par_diff ,dis_impact,acc)

In [ ]:
def fairness_metrics_roc_auc(data , y):
    clf = RandomForestClassifier(n_estimators=10, max_depth=5)

    cv = StratifiedKFold(n_splits=5, random_state=1234, shuffle=True)
    results = pd.DataFrame(columns=['training_score', 'test_score'])

    fprs, tprs, scores, op_list ,op_f_list, od_list ,par_dif_list,dis_impact_list, acc_list = [], [], [], [],[], [], [],[],[]

    start_time = datetime.now()
    for (train, test), i in zip(cv.split(data, y), range(5)):
        clf.fit(data.iloc[train], y.iloc[train])
        _, _, auc_score_train = compute_roc_auc(train,clf, data, y)
        fpr, tpr, auc_score = compute_roc_auc(test,clf, data, y)
        scores.append((auc_score_train, auc_score))
        fprs.append(fpr)
        tprs.append(tpr)
        y_pred = clf.predict(data.iloc[test])
        o_p, o_p_fpr, o_d ,par_dif, dis_impact, acc_ = calc_bias(data.iloc[test],  y.iloc[test], PRIV_FEATURE,y_pred, 'subexperiment - baseline Ensemble'+' model:'+str(i))
        acc_list.append(acc_)
        op_list.append(o_p)
        op_f_list.append(o_p_fpr)
        od_list.append(o_d)
        par_dif_list.append(par_dif)
        dis_impact_list.append(dis_impact)

    std_op = np.std(op_list)
    op_str = f"{mean(op_list):.3f} ± {std_op:.3f}"

    std_op_f = np.std(op_f_list)
    op_f_str = f"{mean(op_f_list):.3f} ± {std_op_f:.3f}"

    std_od = np.std(od_list)
    od_str = f"{mean(od_list):.3f} ± {std_od:.3f}"

    std_par_dif = np.std(par_dif_list)
    par_dif_str = f"{mean(par_dif_list):.3f} ± {std_par_dif:.3f}"

    std_dis_impact = np.std(dis_impact_list)
    dis_imp_str = f"{mean(dis_impact_list):.3f} ± {std_dis_impact:.3f}"


    std_acc = np.std(acc_list)
    acc_str = f"{mean(acc_list):.3f} ± {std_acc:.3f}"
    time_elapsed_baseline = datetime.now() - start_time
    plot_roc_curve(fprs, tprs);
    pd.DataFrame(scores, columns=['AUC Train', 'AUC Test'])
    print('Accaracy list : ', acc_list)
    print('equal opportunity for tpr : ', op_list)
    print('equal opportunity for fpr : ',op_f_list)
    print('odds difference list : ' , od_list)
    print('The mean accuracy is ', acc_str)
    print('The mean equal opportunity for tpr: (close to 0)', op_str)
    print('The mean equal opportunity for fpr: (close to 0)', op_f_str)
    print('The mean odds difference: (close to 0)',od_str)
    print('The mean parity difference: (close to 0)', par_dif_str)
    print('The mean disperate impact: (close to 1)',dis_imp_str)

    return acc_list, op_list, op_f_list ,od_list,par_dif_list , dis_impact_list

In [ ]:
def fairness_metrics_roc_auc_no_print(data , y):
    clf = RandomForestClassifier(n_estimators=10, max_depth=5)

    cv = StratifiedKFold(n_splits=5, random_state=1234, shuffle=True)
    results = pd.DataFrame(columns=['training_score', 'test_score'])

    fprs, tprs, scores, op_list ,op_f_list, od_list ,par_dif_list,dis_impact_list, acc_list = [], [], [], [],[], [], [],[],[]

    start_time = datetime.now()
    for (train, test), i in zip(cv.split(data, y), range(5)):
        clf.fit(data.iloc[train], y.iloc[train])
        _, _, auc_score_train = compute_roc_auc(train,clf, data, y)
        fpr, tpr, auc_score = compute_roc_auc(test,clf, data, y)
        scores.append((auc_score_train, auc_score))
        fprs.append(fpr)
        tprs.append(tpr)
        y_pred = clf.predict(data.iloc[test])
        o_p, o_p_fpr, o_d ,par_dif, dis_impact, acc_ = calc_bias(data.iloc[test],  y.iloc[test], PRIV_FEATURE,y_pred, 'subexperiment - baseline Ensemble'+' model:'+str(i))
        acc_list.append(acc_)
        op_list.append(o_p)
        op_f_list.append(o_p_fpr)
        od_list.append(o_d)
        par_dif_list.append(par_dif)
        dis_impact_list.append(dis_impact)

    return acc_list, op_list, op_f_list ,od_list,par_dif_list , dis_impact_list

## Roc & AUC functions¶


In [ ]:
def plot_roc_curve(fprs, tprs):
    """Plot the Receiver Operating Characteristic from a list
    of true positive rates and false positive rates."""

    # Initialize useful lists + the plot axes.
    tprs_interp = []
    aucs, acc = [], []
    mean_fpr = np.linspace(0, 1, 100)
    f, ax = plt.subplots(figsize=(14,10))

    # Plot ROC for each K-Fold + compute AUC scores.
    for i, (fpr, tpr) in enumerate(zip(fprs, tprs)):
        tprs_interp.append(np.interp(mean_fpr, fpr, tpr))
        tprs_interp[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        ax.plot(fpr, tpr, lw=1, alpha=0.3,
                 label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    # Plot the luck line.
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
             label='Luck', alpha=.8)

    # Plot the mean ROC.
    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=2, alpha=.8)

    # Plot the standard deviation around the mean ROC.
    std_tpr = np.std(tprs_interp, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                     label=r'$\pm$ 1 std. dev.')

    # Fine tune and show the plot.
    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic')
    ax.legend(loc="lower right")
    plt.show()
    return (f, ax)

def compute_roc_auc(index,clf ,data, y):
    y_predict = clf.predict_proba(data.iloc[index])[:,1]
    fpr, tpr, thresholds = roc_curve(y.iloc[index], y_predict)
    auc_score = auc(fpr, tpr)
    return fpr, tpr, auc_score

## Datasets & Params¶


In [ ]:
# choose dataset
#DATASET_NAME = 'ADULT_SEX'
#DATASET_NAME = 'ADULT_RACE'
DATASET_NAME ='COMPAS_RACE_AM'
#DATASET_NAME = 'COMPAS_SEX'
#DATASET_NAME='CREDIT_SEX'
#DATASET_NAME = 'BANK_AGE'


In [ ]:
# upload kaggle.json file using user prompt
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

!mkdir /root/.kaggle/
# Installing the Kaggle package
!pip install kaggle

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"guyrozenblatt","key":"e2ba3aff97edbcc7b8ab392e91dd8c65"}


# creating kaggle.json file with the personal API-Key details
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json


!mkdir ./datasets/compastwoyears/compas-scores-two-years_v1.csv

Saving compas-scores-two-years_v1.csv to compas-scores-two-years_v1 (8).csv
User uploaded file "compas-scores-two-years_v1 (8).csv" with length 2416405 bytes
mv: cannot stat 'kaggle.json': No such file or directory
mkdir: cannot create directory ‘/root/.kaggle/’: File exists
mkdir: cannot create directory ‘./datasets/compastwoyears/compas-scores-two-years_v1.csv’: No such file or directory


In [ ]:
if DATASET_NAME == 'ADULT_SEX':
    TARGET_COL = 'income'
    PRIV_FEATURE = "sex"
    PRIV_CLASS = 1
    PRIV_VALUE_FEATURE = 1
    orig_data = pd.read_csv('/content/adult.csv')
    orig_data[TARGET_COL] = orig_data[TARGET_COL].apply(lambda x: 0 if x == '<=50K' else 1)
    orig_data['sex'] = np.where(orig_data['sex'] == 'Male', 1,0)
    orig_data['race'] = np.where(orig_data['race'] != 'White', 0, 1)
    orig_data.replace('?',np.nan,inplace = True)
    orig_data.dropna(how='any',inplace = True)
    orig_data.drop_duplicates()
    orig_data.reset_index(inplace=True, drop=True)

    privileged_groups = [{'sex': 1}] # Male
    unprivileged_groups = [{'sex': 0}] # Female

    feature_columns = ['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country']

    CATEGORICAL = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'native.country']


elif DATASET_NAME == 'ADULT_RACE':
    TARGET_COL = 'income'
    PRIV_FEATURE = 'race'
    PRIV_CLASS = 1
    PRIV_VALUE_FEATURE = 1
    orig_data = pd.read_csv('/content/adult.csv')
    print(orig_data)
    orig_data[TARGET_COL] = orig_data[TARGET_COL].apply(lambda x: 0 if x == '<=50K' else 1)
    orig_data['sex'] = np.where(orig_data['sex'] == 'Male', 1,0)
    orig_data['race'] = np.where(orig_data['race'] != 'White', 0, 1)

    # orig_data['black'] = np.where(orig_data['race'] == 'Black', 0,1)
    orig_data.replace('?',np.nan,inplace = True)
    orig_data.dropna(how='any',inplace = True)
    orig_data.drop_duplicates()
    orig_data.reset_index(inplace=True, drop=True)

    # feature_columns = ['age', 'workclass', 'fnlwgt', 'education', 'education.num',
    #    'marital.status', 'occupation', 'relationship', 'race', 'sex',
    #    'capital.gain', 'capital.loss', 'hours.per.week', 'native.country', 'black']
    feature_columns = ['age', 'education.num','race', 'sex', 'capital.gain','capital.loss', 'hours.per.week']

    CATEGORICAL = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'native.country']

elif DATASET_NAME =='COMPAS_RACE_AM':
    orig_data = pd.read_csv('/content/compas-scores-two-years_v1.csv')
    orig_data = orig_data[((orig_data['days_b_screening_arrest'] <=30) &
      (orig_data['days_b_screening_arrest'] >= -30) &
      (orig_data['is_recid'] != -1) &
      (orig_data['c_charge_degree'] != 'O') &
      (orig_data['score_text'] != 'N/A')
     )]
    TARGET_COL = 'two_year_recid'
    orig_data['sex'] = np.where(orig_data['sex'] == 'Male', 1, 0)
    orig_data['am'] = np.where(orig_data['race'] == 'African-American', 1,0)
    orig_data['age_cat'] = np.where(orig_data['age_cat'] == 'Greater than 45',45,orig_data['age_cat'])
    orig_data['age_cat'] = np.where(orig_data['age_cat'] == '25 - 45', 25, orig_data['age_cat'])
    orig_data['age_cat'] = np.where(orig_data['age_cat'] == 'Less than 25', 0, orig_data['age_cat'])
    orig_data['c_charge_degree'] = np.where(orig_data['c_charge_degree'] == 'F', 1, 0)
    orig_data['age'] = orig_data['age'].astype('int')
    orig_data['two_year_recid'] = orig_data['two_year_recid'].astype('int')
    PRIV_FEATURE = 'am'
#     PRIV_FEATURE = 'sex'
    PRIV_CLASS = 0
    PRIV_VALUE_FEATURE = 0
    feature_columns = ['age', 'c_charge_degree', 'age_cat', 'sex', 'priors_count',
               'days_b_screening_arrest', 'decile_score','am']
    CATEGORICAL = [ 'age_cat', 'race', 'c_charge_degree', 'score_text']



elif DATASET_NAME =='COMPAS_SEX':
    orig_data = pd.read_csv('../input/compastwoyears/compas-scores-two-years_v1.csv')
    orig_data = orig_data[((orig_data['days_b_screening_arrest'] <=30) &
      (orig_data['days_b_screening_arrest'] >= -30) &
      (orig_data['is_recid'] != -1) &
      (orig_data['c_charge_degree'] != 'O') &
      (orig_data['score_text'] != 'N/A')
     )]
    TARGET_COL = 'two_year_recid'
    orig_data['sex'] = np.where(orig_data['sex'] == 'Male', 1, 0)
    orig_data['am'] = np.where(orig_data['race'] == 'African-American', 1,0)
    PRIV_FEATURE = 'sex'
    PRIV_CLASS = 0
    PRIV_VALUE_FEATURE = 0
    feature_columns = ['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count',
               'days_b_screening_arrest', 'decile_score','am']
    CATEGORICAL = [ 'age_cat', 'race', 'c_charge_degree', 'score_text']


elif DATASET_NAME == 'CREDIT_SEX':
    orig_data = pd.read_csv('../input/german-credit-data-with-risk/german_credit_data.csv')
    TARGET_COL = 'Risk'
    PRIV_FEATURE = 'Sex'
    PRIV_CLASS = 1
    PRIV_VALUE_FEATURE = 0
    orig_data[TARGET_COL] = orig_data[TARGET_COL].apply(lambda x: 0 if x == 'bad' else 1)
    orig_data['Sex'] = np.where(orig_data['Sex'] == 'male', 0,1)
    orig_data = orig_data.drop(["Unnamed: 0"],axis=1)
    orig_data['young'] = orig_data['Age'].apply(lambda x: 0 if x < 25 else 1)
    CATEGORICAL = ['Housing', 'Saving accounts' , 'Checking account', 'Purpose']
    feature_columns = ['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account','Credit amount', 'Duration', 'Purpose']

elif DATASET_NAME =='BANK_AGE':
    orig_data = pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')
    TARGET_COL = 'deposit'
    orig_data[TARGET_COL] = orig_data[TARGET_COL].apply(lambda x: 0 if x == 'no' else 1)

    orig_data['age_cat'] = np.where(((pd.to_numeric(orig_data['age']) > 65) |  (pd.to_numeric(orig_data['age']) < 25)), 0,1)
    PRIV_FEATURE = 'age_cat'
    PRIV_VALUE_FEATURE = 0
    PRIV_CLASS = 1
    feature_columns = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'age_cat',
       'previous', 'poutcome']
    CATEGORICAL = [ 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month','poutcome']

In [ ]:
from sklearn.preprocessing import MinMaxScaler



# dataset for training
orig_data.dropna(thresh=len(feature_columns)/2, inplace=True)
orig_data.reset_index(drop=True, inplace=True)

y_orig = orig_data[TARGET_COL]
# y_orig = orig_data[TARGET_COL].to_frame()

y_orig.dropna(inplace=True)

orig_data = orig_data.loc[:, feature_columns]

scaler = MinMaxScaler()
orig_data = pd.DataFrame(scaler.fit_transform(orig_data),columns = orig_data.columns)
# # data = data[feature_columns]
# encoder = ce.TargetEncoder(cols=CATEGORICAL)

# encoder.fit(orig_data, y_orig)
# orig_data_encoded = encoder.transform(orig_data)

In [ ]:
orig_dataset = pd.concat([orig_data, y_orig], axis=1)
orig_dataset

,age,c_charge_degree,age_cat,sex,priors_count,days_b_screening_arrest,decile_score,am,two_year_recid
0,0.653846,1.0,1.000000,1.0,0.000000,0.483333,0.000000,0.0,0
1,0.205128,1.0,0.555556,1.0,0.000000,0.483333,0.222222,1.0,1
2,0.076923,1.0,0.000000,1.0,0.105263,0.483333,0.333333,1.0,1
3,0.333333,0.0,0.555556,1.0,0.000000,0.500000,0.000000,0.0,0
4,0.294872,1.0,0.555556,1.0,0.368421,0.483333,0.555556,0.0,1
...,...,...,...,...,...,...,...,...,...
6167,0.064103,1.0,0.000000,1.0,0.000000,0.483333,0.666667,1.0,0
6168,0.064103,1.0,0.000000,1.0,0.000000,0.483333,0.222222,1.0,0
6169,0.500000,1.0,1.000000,1.0,0.000000,0.483333,0.000000,0.0,0
6170,0.192308,0.0,0.555556,0.0,0.078947,0.483333,0.111111,1.0,0


In [ ]:
orig_dataset.columns

Index(['age', 'c_charge_degree', 'age_cat', 'sex', 'priors_count',
       'days_b_screening_arrest', 'decile_score', 'am', 'two_year_recid'],
      dtype='object')

# Check original scores

performance for whole ds:subexperiment - baseline Ensemble model:0
conf matrix:
  [[531 142]
 [242 320]] 
accuracy:  0.6890688259109312 precision:  0.6926406926406926 recall:  0.5693950177935944 fpr:  0.21099554234769688 tpr:  0.5693950177935944 fn+fp 384
performance for 0 sub-group:
conf matrix:
  [[322  54]
 [143  90]] 
accuracy:  0.6765188834154351 precision:  0.625 recall:  0.38626609442060084 fpr:  0.14361702127659576 tpr:  0.38626609442060084 fn+fp 197
performance for 1 sub-group:
conf matrix:
  [[209  88]
 [ 99 230]] 
accuracy:  0.7012779552715654 precision:  0.7232704402515723 recall:  0.6990881458966566 fpr:  0.2962962962962963 tpr:  0.6990881458966566 fn+fp 187
bias metrics:
equal opportunity for tpr: (close to 0) 0.31282205147605574
equal opportunity for fpr: (close to 0) 0.15267927501970052
avarage absolute odds difference: (close to 0) 0.23275066324787813
statistical parity difference: (close to 0)  0.37662337662337664
disparate_impact: (close to 1)  0.3116883116883117
per

Accaracy list :  [0.6890688259109312, 0.7125506072874493, 0.6661264181523501, 0.6645056726094003, 0.6839546191247974]
equal opportunity for tpr :  [0.31282205147605574, 0.24638263144940864, 0.3580391434734571, 0.28019336432797975, 0.2793629376827123]
equal opportunity for fpr :  [0.15267927501970052, 0.1904402922118716, 0.2648994840775663, 0.1989608003818203, 0.2537888198757764]
odds difference list :  [0.23275066324787813, 0.2184114618306401, 0.3114693137755117, 0.23957708235490002, 0.2665758787792444]
The mean accuracy is  0.683 ± 0.018
The mean equal opportunity for tpr: (close to 0) 0.295 ± 0.038
The mean equal opportunity for fpr: (close to 0) 0.212 ± 0.042
The mean odds difference: (close to 0) 0.254 ± 0.033
The mean parity difference: (close to 0) 0.402 ± 0.051
The mean disperate impact: (close to 1) 0.198 ± 0.096


In [ ]:
orig_acc ,orig_op_diff, orig_op_diff_fpr, orig_od_diff, orig_par_diff , orig_dis_impact  = fairness_metrics_roc_auc(orig_data, y_orig)

# Find Class & Protected attribute Distribution


In [ ]:
#Based on class
orig_data_one , orig_data_zero = [x for _, x in orig_dataset.groupby(orig_dataset[TARGET_COL] == 0)]

#Based on priv 1
one_one_df, one_zero_df = [x for _, x in orig_data_one.groupby(orig_data_one[PRIV_FEATURE] == 1)]
zero_one_df, zero_zero_df = [x for _, x in orig_data_zero.groupby(orig_data_zero[PRIV_FEATURE] == 1)]

one_one = one_one_df.shape[0]
one_zero = one_zero_df.shape[0]
zero_one = zero_one_df.shape[0]
zero_zero = zero_zero_df.shape[0]

print(zero_zero,zero_one,one_zero,one_one)

1514 1849 1661 1148


# Check SMOTE Scores

In [ ]:
orig_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6172 entries, 0 to 6171
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      6172 non-null   float64
 1   c_charge_degree          6172 non-null   float64
 2   age_cat                  6172 non-null   float64
 3   sex                      6172 non-null   float64
 4   priors_count             6172 non-null   float64
 5   days_b_screening_arrest  6172 non-null   float64
 6   decile_score             6172 non-null   float64
 7   am                       6172 non-null   float64
 8   two_year_recid           6172 non-null   int64  
dtypes: float64(8), int64(1)
memory usage: 434.1 KB


In [ ]:
# def apply_smote(df):
#     df.reset_index(drop=True,inplace=True)
#     cols = df.columns
#     smt = smote(df)
#     df = smt.run()
#     df.columns = cols
#     return df

# orig_dataset = apply_smote(orig_dataset)



In [ ]:
orig_dataset.drop_duplicates()
orig_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6172 entries, 0 to 6171
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      6172 non-null   float64
 1   c_charge_degree          6172 non-null   float64
 2   age_cat                  6172 non-null   float64
 3   sex                      6172 non-null   float64
 4   priors_count             6172 non-null   float64
 5   days_b_screening_arrest  6172 non-null   float64
 6   decile_score             6172 non-null   float64
 7   am                       6172 non-null   float64
 8   two_year_recid           6172 non-null   int64  
dtypes: float64(8), int64(1)
memory usage: 434.1 KB


# Sort these four

In [ ]:
maximum = max(zero_zero,zero_one,one_zero,one_one)
if maximum == zero_zero:
    print("zero_zero is maximum")
if maximum == zero_one:
    print("zero_one is maximum")
if maximum == one_zero:
    print("one_zero is maximum")
if maximum == one_one:
    print("one_one is maximum")

zero_one is maximum


In [ ]:
zero_zero_to_be_incresed = maximum - zero_zero ## where both are 0
one_zero_to_be_incresed = maximum - one_zero ## where class is 1 attribute is 0
zero_one_to_be_incresed = maximum - zero_one ## where class is 1 attribute is 1
one_one_to_be_incresed = maximum - one_one ## where class is 1 attribute is 1


print(zero_zero_to_be_incresed,one_zero_to_be_incresed,zero_one_to_be_incresed, one_one_to_be_incresed)

335 188 0 701


In [ ]:
# def apply_smote(df):
#     df.reset_index(drop=True,inplace=True)
#     cols = df.columns
#     smt = smote(df)
#     df = smt.run()
#     df.columns = cols
#     return df

# orig_dataset = apply_smote(orig_dataset)


In [ ]:

# zero_zero_df['am'] = zero_zero_df['am'].astype(str)
# zero_zero_df['sex'] = zero_zero_df['sex'].astype(str)


# zero_one_df['am'] = zero_one_df['am'].astype(str)
# zero_one_df['sex'] = zero_one_df['sex'].astype(str)

# one_zero_df['am'] = one_zero_df['am'].astype(str)
# one_zero_df['sex'] = one_zero_df['sex'].astype(str)

# one_one_df['am'] = one_one_df['am'].astype(str)
# one_one_df['sex'] = one_one_df['sex'].astype(str)


# def apply_smote(df):
#     df.reset_index(drop=True,inplace=True)
#     cols = df.columns
#     smt = smote(df)
#     df = smt.run()
#     df.columns = cols
#     return df

# orig_dataset = apply_smote(orig_dataset)

zero_zero_df = generate_samples(zero_zero_to_be_incresed,zero_zero_df,'Compas')
one_zero_df = generate_samples(one_zero_to_be_incresed,one_zero_df,'Compas')
zero_one_df = generate_samples(zero_one_to_be_incresed,zero_one_df,'Compas')
one_one_df = generate_samples(one_one_to_be_incresed,one_one_df,'Compas')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature n

In [ ]:
zero_zero_df

,age,c_charge_degree,age_cat,sex,priors_count,days_b_screening_arrest,decile_score,am,two_year_recid
0,0.294872,1.0,0.555556,1.0,0.000000,0.483333,0.333333,1.0,0.0
1,0.089744,1.0,0.555556,1.0,0.078947,0.483333,1.000000,1.0,0.0
2,0.038462,1.0,0.000000,0.0,0.052632,0.483333,0.777778,1.0,0.0
3,0.423077,0.0,1.000000,1.0,0.052632,0.483333,0.000000,1.0,0.0
4,0.423077,1.0,1.000000,0.0,0.184211,0.483333,0.111111,1.0,0.0
...,...,...,...,...,...,...,...,...,...
1844,0.179487,0.0,0.555556,1.0,0.000000,0.483333,0.444444,1.0,0.0
1845,0.256410,1.0,0.555556,1.0,0.236842,0.416667,0.555556,1.0,0.0
1846,0.276923,0.0,0.555556,1.0,0.021053,0.500000,0.555556,1.0,0.0
1847,0.174359,1.0,0.555556,1.0,0.136842,0.496667,0.333333,1.0,0.0


# Append the dataframes

In [ ]:
Upsampled_Dataset = pd.concat([zero_zero_df, zero_one_df, one_zero_df, one_one_df], ignore_index=True)
Upsampled_Dataset = Upsampled_Dataset.reset_index()


In [ ]:
Upsampled_Dataset


,index,age,c_charge_degree,age_cat,sex,priors_count,days_b_screening_arrest,decile_score,am,two_year_recid
0,0,0.294872,1.0,0.555556,1.0,0.000000,0.483333,0.333333,1.0,0.0
1,1,0.089744,1.0,0.555556,1.0,0.078947,0.483333,1.000000,1.0,0.0
2,2,0.038462,1.0,0.000000,0.0,0.052632,0.483333,0.777778,1.0,0.0
3,3,0.423077,0.0,1.000000,1.0,0.052632,0.483333,0.000000,1.0,0.0
4,4,0.423077,1.0,1.000000,0.0,0.184211,0.483333,0.111111,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7391,7391,0.153846,1.0,0.555556,1.0,0.078947,0.483333,0.333333,0.0,1.0
7392,7392,0.017949,0.0,0.000000,1.0,0.047368,0.483333,0.333333,0.0,1.0
7393,7393,0.082051,0.0,0.000000,1.0,0.184211,0.500000,1.000000,0.0,1.0
7394,7394,0.510256,0.0,1.000000,1.0,0.000000,0.513333,0.000000,0.0,1.0


In [ ]:
y_upsampled = Upsampled_Dataset[TARGET_COL]
Upsampled_df_features = Upsampled_Dataset.loc[:, feature_columns]


In [ ]:
y_upsampled

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
7391    1.0
7392    1.0
7393    1.0
7394    1.0
7395    1.0
Name: two_year_recid, Length: 7396, dtype: float64

In [ ]:
Upsampled_df_features

,age,c_charge_degree,age_cat,sex,priors_count,days_b_screening_arrest,decile_score,am
0,0.294872,1.0,0.555556,1.0,0.000000,0.483333,0.333333,1.0
1,0.089744,1.0,0.555556,1.0,0.078947,0.483333,1.000000,1.0
2,0.038462,1.0,0.000000,0.0,0.052632,0.483333,0.777778,1.0
3,0.423077,0.0,1.000000,1.0,0.052632,0.483333,0.000000,1.0
4,0.423077,1.0,1.000000,0.0,0.184211,0.483333,0.111111,1.0
...,...,...,...,...,...,...,...,...
7391,0.153846,1.0,0.555556,1.0,0.078947,0.483333,0.333333,0.0
7392,0.017949,0.0,0.000000,1.0,0.047368,0.483333,0.333333,0.0
7393,0.082051,0.0,0.000000,1.0,0.184211,0.500000,1.000000,0.0
7394,0.510256,0.0,1.000000,1.0,0.000000,0.513333,0.000000,0.0


# Check score after oversampling

performance for whole ds:subexperiment - baseline Ensemble model:0
conf matrix:
  [[500 240]
 [251 489]] 
accuracy:  0.6682432432432432 precision:  0.6707818930041153 recall:  0.6608108108108108 fpr:  0.32432432432432434 tpr:  0.6608108108108108 fn+fp 491
performance for 0 sub-group:
conf matrix:
  [[231 137]
 [ 94 263]] 
accuracy:  0.6813793103448276 precision:  0.6575 recall:  0.7366946778711485 fpr:  0.37228260869565216 tpr:  0.7366946778711485 fn+fp 231
performance for 1 sub-group:
conf matrix:
  [[269 103]
 [157 226]] 
accuracy:  0.6556291390728477 precision:  0.6869300911854104 recall:  0.5900783289817232 fpr:  0.2768817204301075 tpr:  0.5900783289817232 fn+fp 260
bias metrics:
equal opportunity for tpr: (close to 0) 0.14661634888942532
equal opportunity for fpr: (close to 0) 0.09540088826554466
avarage absolute odds difference: (close to 0) 0.12100861857748499
statistical parity difference: (close to 0)  0.09739368998628262
disparate_impact: (close to 1)  0.3711968449931413
perf

Accaracy list :  [0.6682432432432432, 0.6727518593644354, 0.6889790398918187, 0.6795131845841785, 0.6599053414469236]
equal opportunity for tpr :  [0.14661634888942532, 0.09606225361607346, 0.16863753213367616, 0.17424657534246568, 0.22694216446546167]
equal opportunity for fpr :  [0.09540088826554466, 0.14580085059036244, 0.16180756952611935, 0.12526375945138032, 0.19625064201335385]
odds difference list :  [0.12100861857748499, 0.12093155210321795, 0.16522255082989776, 0.14975516739692302, 0.21159640323940776]
The mean accuracy is  0.674 ± 0.010
The mean equal opportunity for tpr: (close to 0) 0.163 ± 0.042
The mean equal opportunity for fpr: (close to 0) 0.145 ± 0.034
The mean odds difference: (close to 0) 0.154 ± 0.034
The mean parity difference: (close to 0) 0.158 ± 0.042
The mean disperate impact: (close to 1) 0.308 ± 0.042


In [ ]:
upsampled_acc, upsampled_op_diff, upsampled_op_diff_fpr, upsampled_od_diff, upsampled_par_diff, upsampled_dis_impact = fairness_metrics_roc_auc(Upsampled_df_features, y_upsampled)

# # Initialize wandb
# wandb.init(project="couple-comparisons")

# Create a bar plot for each couple comparison
fig, ax = plt.subplots(2, 3, figsize=(20, 10), dpi=100)
plt.subplots_adjust(hspace=0.5)
plt.suptitle('Couple Comparisons')

# Set background color
fig.patch.set_facecolor('#f9f9f9')

# Acc vs. Pois_acc plot
ax[0, 0].bar(['orig_acc', 'debias_acc'], [mean(orig_acc), mean(upsampled_acc)], color=['#4e79a7', '#f28e2c'])
ax[0, 0].set_title('Acc vs. debias_acc', fontsize=12, fontweight='bold')
ax[0, 0].set_xlabel('Accuracy', fontsize=10)
ax[0, 0].set_ylabel('Value', fontsize=10)
ax[0, 0].tick_params(axis='both', which='major', labelsize=8)
ax[0, 0].spines['top'].set_visible(False)
ax[0, 0].spines['right'].set_visible(False)

# Op vs. Pois_op plot
ax[0, 1].bar(['orig_op','debias_op'], [mean(orig_op_diff), mean(upsampled_op_diff)], color=['#4e79a7', '#f28e2c'])
ax[0, 1].set_title('Op vs. debias_op', fontsize=12, fontweight='bold')
ax[0, 1].set_xlabel('Equal opportunity for TPR', fontsize=10)
ax[0, 1].set_ylabel('Value', fontsize=10)
ax[0, 1].tick_params(axis='both', which='major', labelsize=8)
ax[0, 1].spines['top'].set_visible(False)
ax[0, 1].spines['right'].set_visible(False)

# Op_f vs. Pois_op_f plotdebias
ax[1, 0].bar(['orig_op_diff_fpr', 'debias_op_diff_fpr'], [mean(upsampled_op_diff_fpr), mean(upsampled_op_diff_fpr)], color=['#4e79a7', '#f28e2c'])
ax[1, 0].set_title('Op_f vs. debias_op_f', fontsize=12, fontweight='bold')
ax[1, 0].set_xlabel('Equal opportunity for FPR', fontsize=10)
ax[1, 0].set_ylabel('Value', fontsize=10)
ax[1, 0].tick_params(axis='both', which='major', labelsize=8)
ax[1, 0].spines['top'].set_visible(False)
ax[1, 0].spines['right'].set_visible(False)

# Od vs. Pois_od plot
ax[1, 1].bar(['orig_od_diff', 'debias_od_diff'], [mean(orig_od_diff), mean(upsampled_od_diff)], color=['#4e79a7', '#f28e2c'])
ax[1, 1].set_title('Od vs. debias_od', fontsize=12, fontweight='bold')
ax[1, 1].set_xlabel('Avarage absolute odds difference', fontsize=10)
ax[1, 1].set_ylabel('Value', fontsize=10)
ax[1, 1].tick_params(axis='both', which='major', labelsize=8)
ax[1, 1].spines['top'].set_visible(False)
ax[1, 1].spines

# Od vs. Pois_od plot
ax[1, 2].bar(['orig_par_diff', 'debias_par_diff'], [mean(orig_par_diff), mean(upsampled_par_diff)], color=['#4e79a7', '#f28e2c'])
ax[1, 2].set_title('orig_par_diff vs. debias_par_diff', fontsize=12, fontweight='bold')
ax[1, 2].set_xlabel('parity diff', fontsize=10)
ax[1, 2].set_ylabel('Value', fontsize=10)
ax[1, 2].tick_params(axis='both', which='major', labelsize=8)
ax[1, 2].spines['top'].set_visible(False)
ax[1, 2].spines

# Od vs. Pois_od plot
ax[0, 2].bar(['orig_dis_impact', 'debias_dis_impact'], [mean(orig_dis_impact), mean(upsampled_dis_impact)], color=['#4e79a7', '#f28e2c'])
ax[0, 2].set_title('orig_dis_impact vs. debias_dis_impact', fontsize=12, fontweight='bold')
ax[0, 2].set_xlabel('Disparate impact', fontsize=10)
ax[0, 2].set_ylabel('Value', fontsize=10)
ax[0, 2].tick_params(axis='both', which='major', labelsize=8)
ax[0, 2].spines['top'].set_visible(False)
ax[0, 2].spines


In [ ]:
# X_train, y_train = df.loc[:, df.columns != 'Probability'], df['Probability']
# X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

# clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR

# print("recall :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'recall'))
# print("far :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'far'))
# print("precision :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'precision'))
# print("accuracy :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'accuracy'))
# print("F1 Score :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'F1'))
# print("aod :"+protected_attribute,measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'aod'))
# print("eod :"+protected_attribute,measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'eod'))

# print("SPD:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'SPD'))
# print("DI:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'DI'))

# Verification

In [ ]:
# first one is class value and second one is protected attribute value
zero_zero = len(Upsampled_Dataset[(Upsampled_Dataset[TARGET_COL] == 0) & (Upsampled_Dataset[PRIV_FEATURE] == 0)])
zero_one = len(Upsampled_Dataset[(Upsampled_Dataset[TARGET_COL] == 0) & (Upsampled_Dataset[PRIV_FEATURE] == 1)])
one_zero = len(Upsampled_Dataset[(Upsampled_Dataset[TARGET_COL] == 1) & (Upsampled_Dataset[PRIV_FEATURE] == 0)])
one_one = len(Upsampled_Dataset[(Upsampled_Dataset[TARGET_COL] == 1) & (Upsampled_Dataset[PRIV_FEATURE] == 1)])

print(zero_zero,zero_one,one_zero,one_one)

1849 1849 1849 1849


In [ ]:
# Record the end time
end_time = time.time()

In [ ]:
# Calculate the elapsed time in minutes
elapsed_time_minutes = (end_time - start_time) / 60
print(f"Elapsed time: {elapsed_time_minutes:.4f} minutes")

Elapsed time: 0.7958 minutes
